In [1]:
from datetime import datetime, timedelta
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import glob

In [2]:
df = pd.read_csv('calicut_sales_clean.csv')

In [3]:
df['date'].min()

'01-01-2013 00:00'

In [4]:
df['date'] = pd.to_datetime(df['date'],format = '%d-%m-%Y %H:%M' ).dt.date

In [5]:
df['date'].min()

datetime.date(2011, 4, 1)

In [6]:
df.tail()

,date,customer_name,item_name,billed_quantity,sale_price,cost_price
753930,2019-12-31,M/S.MALABAR EDUCATIONAL&CHARITABLE TRUST,3M - TRANSBOND XT SYRINGE KIT,1.0,6800.00,6313.0
753931,2019-12-31,"DR.VINU, RAMANATTUKARA",GC TOOTH MOUSSE - VANILLA,0.2,7143.00,5782.0
753932,2019-12-31,DR.PRANEESH PRAHLADHAN,WOODPECKER APEX LOCATOR,1.0,10000.00,8000.0
753933,2019-12-31,"GOVT.DENTAL COLLEGE,CALICUT",SKYDENT UNDER HANGING DENTAL CHAIR (BASIC),2.0,86607.14,72250.0
753934,2019-12-31,"GOVT.DENTAL COLLEGE,CALICUT",MARATHON CLINICAL MICROMOTOR CORD 33 E,2.0,2500.00,2500.0


In [7]:
df = df[df.customer_name != 'M/s. KCK DENTALS DR']
df = df[df.customer_name != 'M/S KCK DENTAL -  TRIVANDRUM']

df = df[df.customer_name != 'KODEN DENTAL (INDIA) PVT.LTD']
df = df[df.customer_name != 'K C K DENTAL(TVM)-DR.']
df = df[df.customer_name != 'KCK EQUIPMENTS CO. PVT. LTD. (DR)']
df = df[df.customer_name != 'KCK DENTAL PVT. LTD. (DR)']
df = df[df.customer_name != 'K C K DENTALS, CALICUT']


In [8]:
len(df.customer_name)

743996

In [9]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [10]:
#df.to_csv('input_data_new.csv', index = False)

In [11]:
df['profit'] = df['billed_quantity']*(df['sale_price'] - df['cost_price'])

In [12]:
df['quarters'] = df.date.dt.quarter
df['year'] = pd.DatetimeIndex(df['date']).year
df['QY']= df['year'].astype(str) + df['quarters'].astype(str)

In [13]:
df.sort_values("QY", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 
qy_val = df.QY.unique() 
qy_val

array(['20112', '20113', '20114', '20121', '20122', '20123', '20124',
       '20131', '20132', '20133', '20134', '20141', '20142', '20143',
       '20144', '20151', '20152', '20153', '20154', '20161', '20162',
       '20163', '20164', '20171', '20172', '20173', '20174', '20181',
       '20182', '20183', '20184', '20191', '20192', '20193', '20194',
       '20201', '20202', '20203', '20204'], dtype=object)

In [14]:
l = len(qy_val)
i=0
while(i<l):
    globals()['df%s' % i] = df[df['QY']==qy_val[i]]
    i = i+1

In [15]:
df0.tail()

,date,customer_name,item_name,billed_quantity,sale_price,cost_price,profit,quarters,year,QY
345226,2011-06-15,DR. MASOOM. WANDOOR,MANI DIAMOND BUR,4.0,57.67,127.00,-277.32,2,2011,20112
345218,2011-06-15,Cash,W-RUBBER BOWL SMALL,5.0,38.45,10.68,138.85,2,2011,20112
345229,2011-06-15,"DR.RAVINDRANATH, TALIPARAMBA",MANI DIAMOND BUR,10.0,57.67,127.00,-693.30,2,2011,20112
345228,2011-06-15,"M/S.M.P.C.DENTAL CLINIC,KODUVALLY.",W-BUR R.A.(CONTRANGLE DIAMOND BUR),6.0,57.67,68.36,-64.14,2,2011,20112
345227,2011-06-15,"M/S.M.P.C.DENTAL CLINIC,KODUVALLY.",MANI DIAMOND BUR,15.0,57.67,127.00,-1039.95,2,2011,20112


In [16]:
scaler = MinMaxScaler()

In [17]:
def order_cluster(cluster_field_name, target_field_name,df_,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df_.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df_,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [18]:
#counts_item = df['item_name'].value_counts().to_dict()
#counts_item

In [19]:
#df_copy = df0
#df_copy['D'] = df_copy['item_name'].map(counts_item)
#df_copy

In [20]:
#df_invoice_freq0

In [21]:
i = 0
while (i<l):
    globals()['df_frequency%s' % i] = globals()['df%s' % i].groupby('item_name').billed_quantity.sum().reset_index()
    globals()['df_frequency%s' % i].columns = ['item_name','frequency']
    globals()['df_profit%s' % i] = globals()['df%s' % i].groupby('item_name').profit.sum().reset_index()
    globals()['df_profit%s' % i].columns = ['item_name','profit']
    
    counts_item = {}
    
    counts_item = globals()['df%s' % i]['item_name'].value_counts().to_dict()
    globals()['df%s' % i]['invoice_freq'] = globals()['df%s' % i]['item_name'].map(counts_item)  
    globals()['df_invoice_freq%s' % i] = globals()['df%s' % i].groupby('item_name').invoice_freq.sum().reset_index()
    globals()['df_invoice_freq%s' % i].columns = ['item_name','invoice_freq']
    
    
    globals()['df_user%s' % i] = pd.merge(globals()['df_profit%s' % i], globals()['df_frequency%s' % i], on='item_name')  
    globals()['df_user%s' % i] = pd.merge(globals()['df_invoice_freq%s' % i], globals()['df_user%s' % i], on='item_name')  
    globals()['df_user%s' % i]['QY'] = qy_val[i]
     
    #globals()['df_user%s' % i][['frequency','profit','invoice_freq']] = scaler.fit_transform(globals()['df_user%s' % i][['frequency','profit','invoice_freq']])
    
    #rank(pct = True) 
    
    globals()['df_user%s' % i]['frequency_cluster'] = globals()['df_user%s' % i].frequency.rank(pct = True) 
    globals()['df_user%s' % i]['profit_cluster'] = globals()['df_user%s' % i].profit.rank(pct = True) 
    globals()['df_user%s' % i]['invoice_freq_cluster'] = globals()['df_user%s' % i].invoice_freq.rank(pct = True) 
    
    #lwknfda
    #kmeans = KMeans(n_clusters=9)
    #kmeans.fit(globals()['df_user%s' % i][['frequency']])
    #globals()['df_user%s' % i]['frequency_cluster'] = kmeans.predict(globals()['df_user%s' % i][['frequency']])
    #globals()['df_user%s' % i] = order_cluster('frequency_cluster', 'frequency',globals()['df_user%s' % i],False)
    
    #kmeans = KMeans(n_clusters=9)
    #kmeans.fit(globals()['df_user%s' % i][['profit']])
    #globals()['df_user%s' % i]['profit_cluster'] = kmeans.predict(globals()['df_user%s' % i][['profit']])
    #globals()['df_user%s' % i] = order_cluster('profit_cluster', 'profit',globals()['df_user%s' % i],False)
    #kmeans = KMeans(n_clusters=9)
    #kmeans.fit(globals()['df_user%s' % i][['invoice_freq']])
    #globals()['df_user%s' % i].dropna()
    #globals()['df_user%s' % i]['invoice_freq_cluster'] = kmeans.predict(globals()['df_user%s' % i][['invoice_freq']])
    #globals()['df_user%s' % i] = order_cluster('invoice_freq_cluster', 'invoice_freq',globals()['df_user%s' % i],False)
    #globals()['df_user%s' % i]['PFI_score'] = globals()['df_user%s' % i]['invoice_freq_cluster'] + globals()['df_user%s' % i]['frequency_cluster'] + 4*globals()['df_user%s' % i]['profit_cluster']
    i= i+1

<ipython-input-21-045b3c9eba36>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df%s' % i]['invoice_freq'] = globals()['df%s' % i]['item_name'].map(counts_item)


In [22]:
df_user20.describe()

,invoice_freq,profit,frequency,frequency_cluster,profit_cluster,invoice_freq_cluster
count,1489.000000,1489.000000,1489.000000,1489.000000,1489.000000,1489.000000
mean,1788.473472,3364.061787,46.900403,0.500336,0.500336,0.500336
std,14809.606075,13217.777313,264.250149,0.287303,0.288771,0.284573
min,1.000000,-14929.350000,0.000000,0.000672,0.000672,0.148758
25%,1.000000,160.000000,2.000000,0.262928,0.250168,0.148758
50%,16.000000,620.000000,6.000000,0.493284,0.500000,0.526528
75%,144.000000,2092.120000,21.000000,0.751175,0.750168,0.749832
max,334084.000000,378142.860000,6868.000000,1.000000,1.000000,1.000000


counts_item = {}
counts_item = df20['item_name'].value_counts().to_dict()
df20['invoice_freq'] = df20['item_name'].map(counts_item)
df20  

df_test = df.groupby('item_name').billed_quantity.sum().reset_index()
df_test.columns = ['item_name', 'billed_quantity']
df_test.sort_values(by = ['billed_quantity'], inplace = True)
df_test.tail(50)

df_test.to_csv('testFreq.csv', index = False)

In [23]:
i = 0
list_of_dataframes = []
while(i<l):
    list_of_dataframes.append(globals()['df_user%s' % i])
    i = i+1

In [24]:
#list_of_dataframes

In [25]:
df_cust = pd.concat(list_of_dataframes)

In [26]:
df_cust.head()

,item_name,invoice_freq,profit,frequency,QY,frequency_cluster,profit_cluster,invoice_freq_cluster
0,3M ADPER EASY ONE,100,-241638.42,10.0,20112,0.570519,0.004550,0.681074
1,3M CAVT-G,729,26170.81,60.0,20112,0.863512,0.963603,0.874886
2,3M CLINPRO SEALNET,49,8753.84,10.0,20112,0.570519,0.888080,0.601911
3,3M CLINPRO TOOTH CREME,81,9726.81,29.0,20112,0.764331,0.898089,0.660146
4,3M CROWN KIT,4,17326.46,2.0,20112,0.240673,0.937216,0.307097


In [27]:
df_cust.describe()

,invoice_freq,profit,frequency,frequency_cluster,profit_cluster,invoice_freq_cluster
count,56703.000000,5.670300e+04,56703.000000,56703.000000,56703.000000,56703.000000
mean,1393.214539,1.299379e+03,48.472410,0.500344,0.500344,0.500344
std,11594.169171,9.333994e+04,251.880132,0.287482,0.288636,0.285030
min,1.000000,-1.361150e+07,-3.000000,0.000505,0.000496,0.118090
25%,1.000000,1.351530e+02,2.000000,0.245902,0.250164,0.155104
50%,16.000000,5.348100e+02,6.000000,0.499588,0.500370,0.504804
75%,121.000000,1.958680e+03,24.000000,0.750270,0.750274,0.749465
max,378225.000000,1.704061e+06,8615.000000,1.000000,1.000000,1.000000


In [28]:
df_cust[df_cust.frequency_cluster > 0.5]

,item_name,invoice_freq,profit,frequency,QY,frequency_cluster,profit_cluster,invoice_freq_cluster
0,3M ADPER EASY ONE,100,-241638.42,10.0,20112,0.570519,0.004550,0.681074
1,3M CAVT-G,729,26170.81,60.0,20112,0.863512,0.963603,0.874886
2,3M CLINPRO SEALNET,49,8753.84,10.0,20112,0.570519,0.888080,0.601911
3,3M CLINPRO TOOTH CREME,81,9726.81,29.0,20112,0.764331,0.898089,0.660146
5,3M EXPRESS STD LIGHT BODY,144,9002.92,13.0,20112,0.624659,0.892630,0.719290
...,...,...,...,...,...,...,...,...
990,WE-CARE LATEX GLOVES - MEDIUM,81,1406.24,22.0,20204,0.886952,0.861554,0.888944
991,WE-CARE LATEX GLOVES - SMALL,2916,9953.57,149.0,20204,0.981076,0.986056,0.991036
992,WHITEROOT PUSH BUTTON SUPERTORQUE HANDPIECE,9,3752.83,3.0,20204,0.508466,0.956175,0.650896
996,WOODPECKER SCALER TIP - G1,1,75.00,3.0,20204,0.508466,0.202191,0.208167


In [29]:
df_cust.QY.unique()

array(['20112', '20113', '20114', '20121', '20122', '20123', '20124',
       '20131', '20132', '20133', '20134', '20141', '20142', '20143',
       '20144', '20151', '20152', '20153', '20154', '20161', '20162',
       '20163', '20164', '20171', '20172', '20173', '20174', '20181',
       '20182', '20183', '20184', '20191', '20192', '20193', '20194',
       '20201', '20202', '20203', '20204'], dtype=object)

In [30]:
dict = {'20111' :'2011Q1', '20112':'2011Q2', '20113':'2011Q3', '20114':'2011Q4',
                
'20121':'2012Q1','20122':'2012Q2', '20123':'2012Q3', '20124':'2012Q4',

'20131':'2013Q1', '20132':'2013Q2','20133':'2013Q3', '20134':'2013Q4', 
                
'20141':'2014Q1', '20142':'2014Q2', '20143':'2014Q3','20144':'2014Q4', 
    
    
'20151':'2015Q1', '20152':'2015Q2', '20153':'2015Q3', '20154':'2015Q4',


'20161':'2016Q1', '20162':'2016Q2', '20163':'2016Q3', '20164':'2016Q4', 

'20171':'2017Q1','20172':'2017Q2', '20173':'2017Q3', '20174':'2017Q4',
                

'20181':'2018Q1', '20182':'2018Q2','20183':'2018Q3', '20184':'2018Q4', 

'20191':'2019Q1', '20192':'2019Q2', '20193':'2019Q3','20194':'2019Q4', 

'20201':'2020Q1', '20202':'2020Q2', '20203':'2020Q3', '20204':'2020Q4',
       'nannan':'2000NA'}

In [31]:
df_cust.replace({"QY": dict}, inplace = True) 

In [32]:
#df_cust['frequency_cluster']=df_cust['frequency_cluster'].replace({0:8, 1:7, 2:6, 3:5, 4:4, 5:3, 6:2, 7:1, 8:0})

In [33]:
#df_cust['profit_cluster']=df_cust['profit_cluster'].replace({0:8, 1:7, 2:6, 3:5, 4:4, 5:3, 6:2, 7:1, 8:0})

In [34]:
df_cust['profit_cluster'] = df_cust['profit_cluster']*100
df_cust['frequency_cluster'] = df_cust['frequency_cluster']*100
df_cust['invoice_freq_cluster'] = df_cust['invoice_freq_cluster']*100

In [35]:
df_cust['Frequency Segment'] = 'Low-Value'
df_cust.loc[df_cust['frequency_cluster'] >= 25,'Frequency Segment'] = 'Below-Average' 
df_cust.loc[df_cust['frequency_cluster'] >= 60,'Frequency Segment'] = 'Average' 
df_cust.loc[df_cust['frequency_cluster'] >= 90,'Frequency Segment'] = 'High-Value' 

In [36]:
df_cust['Profit Segment'] = 'Low-Value'
df_cust.loc[df_cust['profit_cluster'] >= 25,'Profit Segment'] = 'Below-Average' 
df_cust.loc[df_cust['profit_cluster'] >= 60,'Profit Segment'] = 'Average' 
df_cust.loc[df_cust['profit_cluster'] >= 90,'Profit Segment'] = 'High-Value' 

In [37]:
df_cust['Invoice Frequency Segment'] = 'Low-Value'
df_cust.loc[df_cust['profit_cluster'] >= 25,'Invoice Frequency Segment'] = 'Below-Average' 
df_cust.loc[df_cust['profit_cluster'] >= 60,'Invoice Frequency Segment'] = 'Average' 
df_cust.loc[df_cust['profit_cluster'] >= 90,'Invoice Frequency Segment'] = 'High-Value' 

In [38]:
df_cust

,item_name,invoice_freq,profit,frequency,QY,frequency_cluster,profit_cluster,invoice_freq_cluster,Frequency Segment,Profit Segment,Invoice Frequency Segment
0,3M ADPER EASY ONE,100,-241638.42,10.0,2011Q2,57.051865,0.454959,68.107370,Below-Average,Low-Value,Low-Value
1,3M CAVT-G,729,26170.81,60.0,2011Q2,86.351228,96.360328,87.488626,Average,High-Value,High-Value
2,3M CLINPRO SEALNET,49,8753.84,10.0,2011Q2,57.051865,88.808007,60.191083,Below-Average,Average,Average
3,3M CLINPRO TOOTH CREME,81,9726.81,29.0,2011Q2,76.433121,89.808917,66.014559,Average,Average,Average
4,3M CROWN KIT,4,17326.46,2.0,2011Q2,24.067334,93.721565,30.709736,Low-Value,High-Value,High-Value
...,...,...,...,...,...,...,...,...,...,...,...
999,WOODPECKER ULTRASONIC SCALER WITH LED - UDSP,1,1000.00,1.0,2020Q4,15.537849,80.378486,20.816733,Low-Value,Average,Average
1000,X-RAY DEVELOPER LIQUID 250ML - ADARSH,4,112.00,2.0,2020Q4,38.247012,28.137450,51.045817,Below-Average,Below-Average,Below-Average
1001,X-RAY FIXER LIQUID 250ML - ADARSH,4,54.00,2.0,2020Q4,38.247012,15.587649,51.045817,Below-Average,Low-Value,Low-Value
1002,XD-100 MASK,1,266.50,1.0,2020Q4,15.537849,48.705179,20.816733,Low-Value,Below-Average,Below-Average


In [39]:
df_cust = df_cust.round(1)
df_cust.to_csv('calicut_item_quarterwise.csv', index =False)

# combined

In [40]:
#df_cust['twe_customers'] = df_cust['QY'].apply(lambda x : 'Yes' if x > '2019Q4' else 'No') 

In [41]:
df_cust

,item_name,invoice_freq,profit,frequency,QY,frequency_cluster,profit_cluster,invoice_freq_cluster,Frequency Segment,Profit Segment,Invoice Frequency Segment
0,3M ADPER EASY ONE,100,-241638.4,10.0,2011Q2,57.1,0.5,68.1,Below-Average,Low-Value,Low-Value
1,3M CAVT-G,729,26170.8,60.0,2011Q2,86.4,96.4,87.5,Average,High-Value,High-Value
2,3M CLINPRO SEALNET,49,8753.8,10.0,2011Q2,57.1,88.8,60.2,Below-Average,Average,Average
3,3M CLINPRO TOOTH CREME,81,9726.8,29.0,2011Q2,76.4,89.8,66.0,Average,Average,Average
4,3M CROWN KIT,4,17326.5,2.0,2011Q2,24.1,93.7,30.7,Low-Value,High-Value,High-Value
...,...,...,...,...,...,...,...,...,...,...,...
999,WOODPECKER ULTRASONIC SCALER WITH LED - UDSP,1,1000.0,1.0,2020Q4,15.5,80.4,20.8,Low-Value,Average,Average
1000,X-RAY DEVELOPER LIQUID 250ML - ADARSH,4,112.0,2.0,2020Q4,38.2,28.1,51.0,Below-Average,Below-Average,Below-Average
1001,X-RAY FIXER LIQUID 250ML - ADARSH,4,54.0,2.0,2020Q4,38.2,15.6,51.0,Below-Average,Low-Value,Low-Value
1002,XD-100 MASK,1,266.5,1.0,2020Q4,15.5,48.7,20.8,Low-Value,Below-Average,Below-Average


In [42]:
df_cust_date_min = df.groupby('item_name').date.min().reset_index()

In [43]:
df_cust_date_min.columns = ['item_name','date_min']

In [44]:
df_cust_date_max = df.groupby('item_name').date.max().reset_index()

In [45]:
df_cust_date_max.columns = ['item_name','date_max']

In [46]:
df_cust_date = pd.merge(df_cust_date_min, df_cust_date_max, on='item_name')

In [47]:
#df_cust_new_0 = df_cust.groupby('item_name').twe_customers.sum().reset_index()

In [48]:
df_cust_new = df_cust.groupby('item_name').profit_cluster.sum().reset_index()

In [49]:
df_cust_new_2 = df_cust.groupby('item_name').frequency_cluster.sum().reset_index()

In [50]:
df_cust_new_0 = df_cust.groupby('item_name').invoice_freq_cluster.sum().reset_index()

In [51]:
df_cust_comb = pd.merge(df_cust_new, df_cust_new_2, on='item_name')

In [52]:
df_cust_comb = pd.merge(df_cust_comb, df_cust_new_0, on='item_name')

In [53]:
df_cust_comb = pd.merge(df_cust_comb, df_cust_date, on='item_name')

In [54]:
df_cust_comb.profit_cluster = df_cust_comb.profit_cluster/(l)

In [55]:
df_cust_comb.frequency_cluster = df_cust_comb.frequency_cluster/(l)

In [56]:
df_cust_comb.invoice_freq_cluster = df_cust_comb.invoice_freq_cluster/(l)

In [57]:
df_cust_comb.describe()

,profit_cluster,frequency_cluster,invoice_freq_cluster
count,8250.000000,8250.000000,8250.000000
mean,8.817745,8.817493,8.817564
std,13.217653,13.956309,14.357565
min,0.002564,0.002564,0.302564
25%,1.061538,0.802564,0.664103
50%,3.185897,2.737179,2.416667
75%,10.778846,10.544231,10.655128
max,99.205128,99.992308,99.928205


df_cust_comb['profit_cluster'] = df_cust_comb['profit_cluster']*100
df_cust_comb['frequency_cluster'] = df_cust_comb['frequency_cluster']*100
df_cust_comb['invoice_freq_cluster'] = df_cust_comb['invoice_freq_cluster']*100


In [58]:
df_cust_comb.columns = ['item_name','profit_final', 'frequency_final','invoice_freq_final','date_min','date_max']

In [59]:
df_cust_comb

,item_name,profit_final,frequency_final,invoice_freq_final,date_min,date_max
0,1/4*6 MM PRESS NIPPLE,0.664103,1.205128,0.666667,2015-10-12,2016-01-05
1,1/4*8 MM PRESS NIPPLE,1.041026,1.594872,1.543590,2018-03-05,2019-02-23
2,10 INSTRUMENT CASSETTE,2.835897,1.948718,1.807692,2015-02-10,2017-01-05
3,10 INSTRUMENT CASSETTE TRAY,6.343590,3.989744,2.689744,2017-08-30,2020-07-15
4,2-0 VICRYL WITH NEEDLE,1.969231,3.530769,1.248718,2017-03-08,2017-12-27
...,...,...,...,...,...,...
8245,Zzzzz 3M FILTEK Z250 XT NANOHYRBRID B2,1.030769,1.300000,0.664103,2014-05-09,2015-01-26
8246,Zzzzz KODEN LIGHT CURE UNIT (LED),2.333333,1.564103,2.541026,2014-04-29,2015-10-24
8247,Zzzzz MANI PASTE CARRIER 25-40 ASSORTED,0.223077,0.297436,0.397436,2017-11-25,2017-11-25
8248,ZzzzzGC - SOLARE E SCULPT SYRINGE A3,1.087179,5.648718,6.692308,2018-05-16,2019-01-16


In [60]:
df_cust_comb['sales_score'] = 4*df_cust_comb['profit_final'] + df_cust_comb['frequency_final'] + df_cust_comb['invoice_freq_final']
df_cust_comb['sales_score'] = df_cust_comb.sales_score.rank(pct = True) 
df_cust_comb['sales_score'] = df_cust_comb['sales_score']*100

df_cust_comb['profit_final'] = df_cust_comb.sales_score.rank(pct = True) 
df_cust_comb['profit_final'] = df_cust_comb['profit_final']*100

df_cust_comb['frequency_final'] = df_cust_comb.sales_score.rank(pct = True) 
df_cust_comb['frequency_final'] = df_cust_comb['frequency_final']*100

df_cust_comb['invoice_freq_final'] = df_cust_comb.sales_score.rank(pct = True) 
df_cust_comb['invoice_freq_final'] = df_cust_comb['invoice_freq_final']*100

In [61]:
df_cust_comb['Frequency Segment'] = 'Low-Value'
df_cust_comb.loc[df_cust_comb['frequency_final'] >= 25,'Frequency Segment'] = 'Below-Average' 
df_cust_comb.loc[df_cust_comb['frequency_final'] >= 60,'Frequency Segment'] = 'Average' 
df_cust_comb.loc[df_cust_comb['frequency_final'] >= 90,'Frequency Segment'] = 'High-Value' 

In [62]:
df_cust_comb['Profit Segment'] = 'Low-Value'
df_cust_comb.loc[df_cust_comb['profit_final'] >= 25,'Profit Segment'] = 'Below-Average' 
df_cust_comb.loc[df_cust_comb['profit_final'] >= 60,'Profit Segment'] = 'Average' 
df_cust_comb.loc[df_cust_comb['profit_final'] >= 90,'Profit Segment'] = 'High-Value' 

In [63]:
df_cust_comb['Invoice Frequency Segment'] = 'Low-Value'
df_cust_comb.loc[df_cust_comb['invoice_freq_final'] >= 25,'Invoice Frequency Segment'] = 'Below-Average' 
df_cust_comb.loc[df_cust_comb['invoice_freq_final'] >= 60,'Invoice Frequency Segment'] = 'Average' 
df_cust_comb.loc[df_cust_comb['invoice_freq_final'] >= 90,'Invoice Frequency Segment'] = 'High-Value' 

In [64]:
df_cust_comb['Sales Segment'] = 'Low-Value'
df_cust_comb.loc[df_cust_comb['sales_score'] >= 25,'Sales Segment'] = 'Below-Average' 
df_cust_comb.loc[df_cust_comb['sales_score'] >= 60,'Sales Segment'] = 'Average' 
df_cust_comb.loc[df_cust_comb['sales_score'] >= 90,'Sales Segment'] = 'High-Value' 

In [65]:
df_cust_comb[df_cust_comb['invoice_freq_final'] > 60]

,item_name,profit_final,frequency_final,invoice_freq_final,date_min,date_max,sales_score,Frequency Segment,Profit Segment,Invoice Frequency Segment,Sales Segment
3,10 INSTRUMENT CASSETTE TRAY,61.248485,61.248485,61.248485,2017-08-30,2020-07-15,61.248485,Average,Average,Average,Average
15,3M - 1860 N95 RESPIRATOR,71.872727,71.872727,71.872727,2019-06-03,2020-10-30,71.872727,Average,Average,Average,Average
17,3M - DURELON INTRODUCTORY PACK,66.884848,66.884848,66.884848,2016-05-24,2018-06-19,66.884848,Average,Average,Average,Average
18,3M - ELIPAR DEEP CURE - L RESTORATIVE,89.830303,89.830303,89.830303,2016-05-24,2020-06-17,89.830303,Average,Average,Average,Average
28,3M - TRANSBOND XT SYRINGE KIT,83.600000,83.600000,83.600000,2017-08-10,2020-10-10,83.600000,Average,Average,Average,Average
...,...,...,...,...,...,...,...,...,...,...,...
8227,ZzzADENTA CONTRA ANGEL BUR,70.254545,70.254545,70.254545,2017-04-13,2018-03-26,70.254545,Average,Average,Average,Average
8231,ZzzD-TEH CEM-ZINC,70.000000,70.000000,70.000000,2017-04-10,2018-07-10,70.000000,Average,Average,Average,Average
8234,ZzzRK - IMPRESSION TRAY BRASS,72.169697,72.169697,72.169697,2017-04-05,2018-04-20,72.169697,Average,Average,Average,Average
8241,ZzzzPREMADENT 6'S,61.321212,61.321212,61.321212,2017-04-04,2017-12-16,61.321212,Average,Average,Average,Average


In [66]:
df_cust_comb = df_cust_comb.round(1)

In [67]:
df_cust_comb.to_csv('final.csv', index =False)